In [1]:
%load_ext autoreload
%autoreload 2

# 0. Intro
1. 利用InferNet生成GT-2需要的params，由此可以计算MDN的input
# 1. preparations
## 1.1 全局设置

In [6]:
import sys
sys.path.insert(0, 'D:\\Desktop\\PROJ\\PAProj')

In [7]:
import torch
import os
import torch.nn as nn
import numpy as np
import random
from MLP.utils import setup_seed
from MLP.utils import load_checkpoint
from MLP.utils import get_InferNet_save_path
import pandas as pd
from tqdm import tqdm
from MLP.models import MLP_2_1
from importlib import reload
from MLP.my_collate_fn import my_collate_fn_GT2,my_collate_fn_3
from torch.utils.data import DataLoader, SequentialSampler, SubsetRandomSampler
from MLP.mydataset_GT2 import myDataset

from MLP.Config import config_GT_gen
reload(config_GT_gen)  # 必须reload！！
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

bound_alpha = torch.tensor([-0.3,0.3],device=device)
bound_labda = torch.tensor([0.01,18],device=device)

opt = config_GT_gen.DefaultConfig()

## 1.2 相关path

In [8]:
MODEL_NAME = "InferNet_GT2"

In [12]:
setup_seed(opt.seed)

dataset = myDataset(opt.train_path, opt.target_path_metric, opt.target_path_loss, opt.params_opitim_path,opt.data_key_path, opt.NLL_metric_path)

data_idx = np.arange(0,dataset.__len__()).tolist()
print("seed=",opt.seed)
print("arti=",opt.ARTIFICIAL)
print("GT_w_Params=",opt.GT_w_Params)

MODEL_NAME = "InferNet_GT" +str(opt.GT_w_Params)
print("MODEL_NAME=",MODEL_NAME)

seed= 62
arti= False
GT_w_Params= 2
MODEL_NAME= InferNet_GT2


# 2. dataloader
1. 每次返回1个data file，并且按照字典序

In [13]:
data_loader = DataLoader(dataset=dataset, batch_size=1, shuffle=False, num_workers=0, drop_last=False, sampler=SequentialSampler(data_idx), collate_fn=my_collate_fn_GT2)

# 3. 读取model并且生成

- 读取model

In [15]:
mlp = MLP_2_1()
if opt.seed in [4,31,204,35,407]:
    mlp.load_state_dict(torch.load(opt.model_params_MLP))
else:
    model_path = get_InferNet_save_path(opt.ARTIFICIAL, opt.seed, opt.net_root_path,opt.noise_pct,MODEL_NAME)
    mlp, _ = load_checkpoint(model_path, mlp)
mlp = mlp.to(device=device)

In [16]:
params_arr = np.array([0.]*(dataset.__len__()*2)).reshape(-1,2)
params = pd.DataFrame(params_arr,columns = ['alpha','labda'])

In [17]:
model=mlp.eval()
for batch_id, data in enumerate(data_loader):

    input_data, _,_, _, _, _= data
    # Do the inference
    input_data = input_data.to(device)

    Alpha, Labda = mlp(input_data)

    Alpha = Alpha.detach().cpu().numpy()
    Labda = Labda.detach().cpu().numpy()

    params.loc[batch_id,'alpha'] = Alpha
    params.loc[batch_id,'labda'] = Labda

print("Done")
print(params.head())

Done
      alpha     labda
0  0.003386  0.302196
1  0.011722  0.413244
2 -0.001566  0.353366
3  0.015664  0.446528
4  0.016598  0.581930


- save

In [18]:
print(opt.params_gen_path)

D:/Desktop/PROJ/PAProj\data/SA_PT/params_GT2_seed=62.csv


In [19]:
params.to_csv(opt.params_gen_path,encoding="utf-8", header=True,index=False)